##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 메트릭 및 옵티마이저 마이그레이션

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/metrics_optimizers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

TF1에서 `tf.metrics`는 모든 메트릭 함수에 대한 API 네임스페이스입니다. 각 메트릭은 `label`과 `prediction`을 입력 매개변수로 사용하고 해당 메트릭 텐서를 결과로 반환하는 함수입니다. TF2에서 `tf.keras.metrics`는 모든 메트릭 함수와 객체를 포함합니다. `Metric` 객체는 `tf.keras.Model`과 `tf.keras.layers.layer`와 함께 사용하여 메트릭 값을 계산할 수 있습니다.

## 설치하기

몇 가지 필요한 TensorFlow 가져오기로 시작합니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

그리고 데모용으로 몇 가지 간단한 데이터를 준비합니다.

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [0, 0, 1]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [0, 1, 1]

## TF1: Estimator를 사용하는 tf.compat.v1.metrics

TF1에서 메트릭은 `eval_metric_ops`로 `EstimatorSpec`에 추가될 수 있으며 연산은 `tf.metrics`에 정의된 모든 메트릭 함수를 통해 생성됩니다. 예제에 따라 `tf.metrics.accuracy`를 사용하는 방법을 확인할 수 있습니다.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(2)(features)
  predictions = tf.math.argmax(input=logits, axis=1)
  loss = tf1.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  accuracy = tf1.metrics.accuracy(labels=labels, predictions=predictions)
  return tf1.estimator.EstimatorSpec(mode, 
                                     predictions=predictions,
                                     loss=loss, 
                                     train_op=train_op,
                                     eval_metric_ops={'accuracy': accuracy})

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

In [ ]:
estimator.evaluate(_eval_input_fn)

또한, `tf.estimator.add_metrics()`를 통해 메트릭을 Estimator에 직접 추가할 수 있습니다.

In [ ]:
def mean_squared_error(labels, predictions):
  labels = tf.cast(labels, predictions.dtype)
  return {"mean_squared_error": 
          tf1.metrics.mean_squared_error(labels=labels, predictions=predictions)}

estimator = tf1.estimator.add_metrics(estimator, mean_squared_error)
estimator.evaluate(_eval_input_fn)

## TF2: tf.keras.Model을 사용하는 Keras 메트릭 API

TF2에서 `tf.keras.metrics`는 모든 메트릭 클래스와 함수를 포함합니다. 이는 OOP 스타일로 설계되어 있으며 다른 `tf.keras` API와 긴밀하게 통합합니다. 모든 메트릭은 `tf.keras.metrics` 네임스페이스에서 찾을 수 있으며 일반적으로 `tf.compat.v1.metrics`와 `tf.keras.metrics` 사이에서 직접 매핑합니다.

다음 예제에서 메트릭이 `model.compile()` 메서드에 추가됩니다. 사용자는 레이블 및 예측 텐서를 지정하지 않고 메트릭 인스턴스만 생성하면 됩니다. Keras 모델은 모델 출력과 레이블을 메트릭 객체로 라우팅합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

inputs = tf.keras.Input((2,))
logits = tf.keras.layers.Dense(2)(inputs)
predictions = tf.math.argmax(input=logits, axis=1)
model = tf.keras.models.Model(inputs, predictions)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

Eager 실행을 사용하도록 설정하면 `tf.keras.metrics.Metric` 인스턴스를 직접 사용하여 numpy 데이터 또는 Eager 텐서를 평가할 수 있습니다. `tf.keras.metrics.Metric` 객체는 상태 저장 컨테이너입니다. 메트릭 값은 `metric.update_state(y_true, y_pred)`를 통해 업데이트할 수 있으며 결과는 `metrics.result()`로 검색할 수 있습니다.


In [ ]:
accuracy = tf.keras.metrics.Accuracy()

accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 1])
accuracy.result().numpy()


In [ ]:
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 0])
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[1, 1, 0, 0])
accuracy.result().numpy()

`tf.keras.metrics.Metric`에 대한 자세한 내용은 `tf.keras.metrics.Metric`의 API 설명서와 [마이그레이션 가이드](https://www.tensorflow.org/guide/effective_tf2#new-style_metrics_and_losses)를 참조하세요.

## TF1.x 옵티마이저를 Keras 옵티마이저로 마이그레이션하기

[Adam 옵티마이저](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer)와 [경사 하강 옵티마이저](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/GradientDescentOptimizer)와 같은 `tf.compat.v1.train`의 옵티마이저는 `tf.keras.optimizers`에서 동일한 기능을 합니다.

아래 표에는 이러한 레거시 옵티마이저를 Keras에 맞게 변환하는 방법이 요약되어 있습니다. [기본 학습률 업데이트하기](../../guide/effective_tf2.ipynb#optimizer_defaults)와 같은 추가 단계가 필요하지 않는 한 TF1.x 버전을 TF2 버전으로 직접 교체할 수 있습니다.

옵티마이저를 변환하면 [이전 체크포인트가 호환되지 않을 수 있습니다](./migrating_checkpoints.ipynb).

<table>
  <tr>
    <th>TF1.x</th>
    <th>TF2</th>
    <th>추가 단계</th>
  </tr>
  <tr>
    <td>`tf.v1.train.GradientDescentOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>없음</td>
  </tr>
  <tr>
    <td>`tf.v1.train.MomentumOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>`momentum` 인수 포함</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdamOptimizer`</td>
    <td>`tf.keras.optimizers.Adam`</td>
    <td>`beta1`과 `beta2` 인수를 `beta_1`과 `beta_2`로 이름 변경</td>
  </tr>
  <tr>
    <td>`tf.v1.train.RMSPropOptimizer`</td>
    <td>`tf.keras.optimizers.RMSprop`</td>
    <td>`decay` 인수를 `rho`로 이름 변경</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdadeltaOptimizer`</td>
    <td>`tf.keras.optimizers.Adadelta`</td>
    <td>없음</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdagradOptimizer`</td>
    <td>`tf.keras.optimizers.Adagrad`</td>
    <td>없음</td>
  </tr>
  <tr>
    <td>`tf.v1.train.FtrlOptimizer`</td>
    <td>`tf.keras.optimizers.Ftrl`</td>
    <td>`accum_name`와 `linear_name` 인수 제거</td>
  </tr>
  <tr>
    <td>`tf.contrib.AdamaxOptimizer`</td>
    <td>`tf.keras.optimizers.Adamax`</td>
    <td>`beta1`과 `beta2` 인수를 `beta_1`과 `beta_2`로 이름 변경</td>
  </tr>
  <tr>
    <td>`tf.contrib.Nadam`</td>
    <td>`tf.keras.optimizers.Nadam`</td>
    <td>`beta1`과 `beta2` 인수를 `beta_1`과 `beta_2`로 이름 변경</td>
  </tr>
</table>

참고: TF2에서 모든 엡실론(수치 안정 상수)은 이제 `1e-8` 대신 `1e-7`로 기본 설정됩니다. 이 차이는 대부분의 사용 사례에서 무시할 수 있습니다.